# 

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder \
    .appName("HadoopCatalogExample") \
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.my_catalog.type", "hadoop") \
    .config("spark.sql.catalog.my_catalog.warehouse", "file:///iceberg/warehouse/") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/30 20:09:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
# spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
devices = spark.read.option("header","true").csv("/home/iceberg/data/devices.csv")

df = events.join(devices,on="device_id",how="left")
df = df.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family'})

df.show()

+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|     

In [4]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
sortedTwo.show()


+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  device_id|    user_id|            referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  532630305| 1129583063|                NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|         Other|    Other|             Other|
| 1088283544| -648945006|                NULL|    www.eczachly.com|                   /|2021-01-07 02:58:...|2021-01-07 00:00:00|      PetalBot|  Android|Generic Smartphone|
| -158310583|-1871780024|                NULL|    www.eczachly.com|                   /|2021-01-07 04:17:...|2021-01-07 00:00:00| 

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  device_id|    user_id|            referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
| -643696601| 1272828233|                NULL|admin.zachwilson....|                   /|2021-01-02 13:53:...|2021-01-02 00:00:00|        Chrome|  Windows|             Other|
|  532630305|  747494706|                NULL|admin.zachwilson....|                   /|2021-01-02 19:36:...|2021-01-02 00:00:00|         Other|    Other|             Other|
|  898871897| 2110046626|                NULL|admin.zachwilson....|       /wp-login.php|2021-01-02 19:57:...|2021-01-02 00:00:00| 

In [ ]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

In [5]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#261, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=410]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters:

In [10]:
!pip show jupysql ipython-sql pyspark pyiceberg

Name: jupysql
Version: 0.10.12
Summary: Better SQL in Jupyter
Home-page: https://github.com/ploomber/jupysql
Author: Ploomber
Author-email: contact@ploomber.io
License: None
Location: /usr/local/lib/python3.8/dist-packages
Requires: ploomber-core, sqlglot, jupysql-plugin, ipython, sqlparse, jinja2, prettytable, sqlalchemy, ploomber-extension, ipython-genutils
Required-by: 
---
Name: ipython-sql
Version: 0.5.0
Summary: RDBMS access via IPython
Home-page: https://github.com/catherinedevlin/ipython-sql
Author: Catherine Devlin
Author-email: catherine.devlin@gmail.com
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: sqlalchemy, six, ipython-genutils, ipython, sqlparse, prettytable
Required-by: 
---
Name: pyspark
Version: 3.5.6
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.8/di

In [11]:
!pip install posthog==2.5.0 --force-reinstall

  Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Using cached charset_normalizer-3.4.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (147 kB)
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: certifi
    Found existing installation: certifi 2025.6.15
    Uninstalling certifi-2025.6.15:
      Successfully uninstalled certifi-2025.6.15
  Attempting uninstall: urllib3
    Foun

In [10]:
!pip install jupysql==0.10.13 --force-reinstall

     |████████████████████████████████| 95 kB 154 kB/s eta 0:00:01
  Using cached sqlalchemy-2.0.41-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached jupysql_plugin-0.4.5-py3-none-any.whl (192 kB)
  Using cached ploomber_core-0.2.26-py3-none-any.whl (22 kB)
  Using cached prettytable-3.11.0-py3-none-any.whl (28 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached sqlglot-26.31.0-py3-none-any.whl (474 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached ipython-8.12.0-py3-none-any.whl (796 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
  Using cached greenlet-3.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (650 kB)
  Using cached PyYAML-6.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (746 kB)
  Using cached posthog-4.2.0-py2.py3-none-any.whl (96 kB)
  Using cached wcwidth-0.2.13-py2.py3-none-any.whl

In [12]:
!pip show posthog

Name: posthog
Version: 2.5.0
Summary: Integrate PostHog into any python application.
Home-page: https://github.com/posthog/posthog-python
Author: Posthog
Author-email: hey@posthog.com
License: MIT License
Location: /usr/local/lib/python3.8/dist-packages
Requires: python-dateutil, backoff, requests, monotonic, six
Required-by: ploomber-core


In [6]:
%load_ext sql

ImportError: cannot import name 'escape_string_literals_with_colon_prefix' from 'sql.parse' (/usr/local/lib/python3.8/dist-packages/sql/parse.py)

In [3]:
%%sql spark --spark-session spark

UsageError: Cell magic `%%sql` not found.


In [18]:
spark.sql("CREATE DATABASE IF NOT EXISTS bootcamp5")


DataFrame[]

In [6]:
spark.sql("DROP TABLE IF EXISTS bootcamp.events")


DataFrame[]

In [22]:
spark.sql("DROP TABLE IF EXISTS bootcamp.events_sorted")


DataFrame[]

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergTest") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "file:///tmp/iceberg_warehouse") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()


In [20]:
spark.sql("""
CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date)
""")

DataFrame[]

In [23]:
spark.sql("SELECT * FROM bootcamp.events")

DataFrame[url: string, referrer: string, browser_family: string, os_family: string, device_family: string, host: string, event_time: timestamp, event_date: date]

In [16]:
%%sparksql --output text

UsageError: Cell magic `%%sparksql` not found.


In [15]:
spark.sql("""
CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date))
""")


DataFrame[]

In [24]:
df = spark.sql("DESCRIBE TABLE EXTENDED bootcamp.events")
df.show(truncate=False)


+----------------------------+-----------------------+-------+
|col_name                    |data_type              |comment|
+----------------------------+-----------------------+-------+
|url                         |string                 |NULL   |
|referrer                    |string                 |NULL   |
|browser_family              |string                 |NULL   |
|os_family                   |string                 |NULL   |
|device_family               |string                 |NULL   |
|host                        |string                 |NULL   |
|event_time                  |timestamp              |NULL   |
|event_date                  |date                   |NULL   |
|# Partition Information     |                       |       |
|# col_name                  |data_type              |comment|
|event_date                  |date                   |NULL   |
|                            |                       |       |
|# Metadata Columns          |                       | 

In [30]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_date));

UsageError: Cell magic `%%sql` not found.


In [26]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [121]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
2896920,4,sorted
3211534,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
